## Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
import matplotlib.pyplot as plt
import pandas as pd
import csv
import itertools


## Get data

In [21]:
csv_file = 'LyricsFind.csv'
se_values = []

nb_song = 50
# Préparer les données textuelles pour l'entraînement d'un modèle de langue.
# Commencer par lire un fichier et décompose son texte en phrases distinctes.
with open(csv_file, 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    tmp_cpt = 0
    for row in reader:
        if tmp_cpt > nb_song :
            continue
        tmp_cpt = tmp_cpt + 1
        se_values.append(row['seq'].split('\n'))

lyrics = list(itertools.chain.from_iterable(se_values))
len(lyrics)

2153

## Data Preprocessing

In [17]:

#La classe Keras Tokenizer, apprend le vocabulaire à partir des phrases d'entrée,
# tokenise les données textuelles une fois qu'elles ont été analysées.

# Tokenize the text data
tokenizer = Tokenizer()
#fit_on_texts Updates internal vocabulary based on a list of texts
tokenizer.fit_on_texts(lyrics)
total_words = len(tokenizer.word_index) + 1
print(total_words)
# Les données tokenisées sont ensuite utilisées pour produire des séquences de n-grammes,
# chaque séquence contient une plage de tokens du début à l'index actuel.
# Input_sequences contient une liste de ces séquences.
# Les séquences sont complétées par des zéros afin de garantir une longueur uniforme.

# Create input sequences
input_sequences = []
for line in lyrics:
  #texts_to_sequences Transforms each text in texts to a sequence of integers.
  #it basically takes each word in the text and replaces it with its
  # corresponding integer value from the word_index dictionary.
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(0, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


# Pad sequences and split into predictors and label
# pad_sequences is used to ensure that all sequences in a list have the same length.
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

predictors, labels = input_sequences[:, :-1], input_sequences[:, -1]

print(predictors.shape)
print(type(labels))

1514
(11089, 15)
<class 'numpy.ndarray'>


## Model building

In [4]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')


## Model training & evaluation

In [5]:
model.fit(predictors, tf.keras.utils.to_categorical(labels), epochs=100)


Epoch 1/100
347/347 [==============================] - 11s 22ms/step - loss: 6.2103
Epoch 2/100
347/347 [==============================] - 7s 21ms/step - loss: 5.8025
Epoch 3/100
347/347 [==============================] - 8s 23ms/step - loss: 5.4390
Epoch 4/100
347/347 [==============================] - 8s 23ms/step - loss: 4.9922
Epoch 5/100
347/347 [==============================] - 9s 25ms/step - loss: 4.5628
Epoch 6/100
347/347 [==============================] - 8s 24ms/step - loss: 4.1802
Epoch 7/100
347/347 [==============================] - 9s 25ms/step - loss: 3.8348
Epoch 8/100
347/347 [==============================] - 9s 26ms/step - loss: 3.5276
Epoch 9/100
347/347 [==============================] - 9s 26ms/step - loss: 3.2561
Epoch 10/100
347/347 [==============================] - 9s 26ms/step - loss: 3.0118
Epoch 11/100
347/347 [==============================] - 9s 26ms/step - loss: 2.7992
Epoch 12/100
347/347 [==============================] - 9s 27ms/step - loss: 2.6056


## Lyrics Generation

In [12]:
def pred_class(text): 
    seed_text = text
    next_words = 50
    
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    
    return seed_text


In [20]:
# lancement du chatbot
while True:
    message = input("")
    intents = pred_class(message)
    print(intents)

 there is


there is something else i don't care to fake it with no games we had do you no never things i'm ha gone through my life in control that you take me it all my fault that your'e high that it all that it all that and all your time lights like
